In [1]:
import sys
import pandas as pd
sys.path.append('../')
from src.EspnFantasyMatchUp import EspnFantasyMatchUp
from src.utils.io_json import read_json
# from src.utils.utils import filter_by_round_stats
# from src.utils.utils import simple_matchup_projection

# Input variables

In [2]:
week = 15  # Upcoming round number of the fantasy league to project: CHANGE THIS ACCORDINGLY
start_date = '2022-01-24'  # start date of the week to project: CHANGE THIS ACCORDINGLY
team1 = 'DGGG'  # Abbreviation of fantasy team 1: CHANGE THIS ACCORDINGLY
team2 = 'CHAL'  # Abbreviation of fantasy team 2: CHANGE THIS ACCORDINGLY

start_date = pd.to_datetime(start_date)
end_date = start_date + pd.Timedelta(6, 'D')  # Usually a round lasts for a week, if this is not the case, change end data manually.
print(end_date)

2022-01-30 00:00:00


# Fetch Data

In [3]:
settings = read_json('../config/config.json')

In [4]:
cookies = settings['cookies']
league_settings = settings['league']
url_settings = {
    'url_fantasy': settings['url']['fantasy_league'],
    'url_nba': settings['url']['nba']
}

In [5]:
espn_api = EspnFantasyMatchUp(
    week, team1, team2,
    cookies=cookies, **url_settings, **league_settings
)

In [6]:
espn_api.naive_matchup_projection()

18 teams participating


TypeError: get_total_team_stats_upto_round() takes 1 positional argument but 2 were given

In [ ]:
teams_data = espn_api.get_teams_data()

In [ ]:
schedule_df = espn_api.get_schedule_data(since=start_date, to=end_date)

# Get Weekly Schedule of Match-up
Number of games played by the two fantasy teams in the upcoming round, Injured players are excluded, DTD are included.

In [ ]:
table, total = espn_api.compare_schedules(teams_data, schedule_df, team1, team2)

In [ ]:
pd.concat((table, total.to_frame('Total').T), axis=0)

# Project Upcoming Match-up Stats

In [ ]:
fantasy_teams_current_stats_df = espn_api.make_stat_table(teams_data)

In [ ]:
fantasy_teams_stats_df = espn_api.get_total_team_stats_upto_round(teams_data, week=week)

## Naive projection
Based on teams' total stats

In [ ]:
simple_matchup_projection(fantasy_teams_stats_df, team1, team2)

## Simulation Projection
Based on teams' schedule, players and their mean stats.

In [ ]:
espn_api.simulation(teams_data, schedule_df, team1, team2)

# Scenarios

In [ ]:
stats_per_round = espn_api.get_fantasy_team_stats_per_round(teams_data)
stats_current_round = filter_by_round_stats(stats_per_round, week=week, stats=espn_api.simulation_stats)

In [ ]:
espn_api.stat_type_code = '002022'
scn_schedule_df = espn_api.get_schedule_data(since=pd.to_datetime('2022-01-30'), to=end_date)

In [ ]:
espn_api.simulation(teams_data, scn_schedule_df, team1, team2, 
                    current_score=stats_current_round)

In [ ]:
# add = {'home': {'<Name>': [<Empty> or <dates>]}, 'away': 
# add = {'home': {'DeAndre Jordan': []}, 'away': {}}
# add = {'home': {'Rudy Gay': []}, 'away': {}}
# add = {"home": {'Nicolas Batum': []}, "away": {}}
add = {'home': {"Keifer Sykes": []}, 'away': {}}
ignore = {'home': {}, 'away': {}}

In [ ]:
espn_api.simulation(teams_data, scn_schedule_df, team1, team2, current_score=stats_current_round, ignore_players=ignore, add_players=add)

In [ ]:
ign = ['Nicolas Batum', 'Rudy Gay', 'Dorian Finney-Smith']

In [ ]:
df = []
stat_type_code = '032022'
for i in ign:
    ignore = {'home': {i: []}, 'away': {}}
    df_tmp = espn_api.simulation(teams_data, scn_schedule_df, team1, team2, 
                                 current_score=stats_current_round, ignore_players=ignore, add_players=add)
    df.append(df_tmp)

In [ ]:
# espn_api.stat_type_code = '002022'
stat_type_code = '032022'
# df1 = espn_api.simulation(teams_data, scn_schedule_df, team1, team2, 
#                     current_score=stats_current_round, ignore_players=ignore, add_players=add)

In [ ]:
df = pd.concat(df, axis=1)

In [ ]:
df.columns = pd.MultiIndex.from_product([ign, df_tmp.columns])

In [ ]:
df